In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/email_bert
!mkdir -p /content/drive/MyDrive/email_bert/datasets
!mkdir -p /content/drive/MyDrive/email_bert/data
!mkdir -p /content/drive/MyDrive/email_bert/tokenizer
!mkdir -p /content/drive/MyDrive/email_bert/checkpoints

!pip install transformers datasets tokenizers pandas
# (you don't actually need `email-parser` – you're using Python's built-in `email` module)


Mounted at /content/drive


In [ ]:

# Optional: keep a backup in Drive
!cp  /content/drive/MyDrive/email_bert/datasets/enron_mail_20150507.tar.gz /content/enron_mail_20150507.tar.gz
!tar -xzf /content/enron_mail_20150507.tar.gz -C /content/

In [ ]:

# Extract the downloaded tarball
enron_dir = '/content/maildir/'


# Data processing: Process email data instead of movie dialogues
import os
import email
import pandas as pd
from pathlib import Path
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam
from email.parser import Parser
import json

def extract_email_content(email_path):
    """Extract content from an email file"""
    try:
        with open(email_path, 'r', encoding='latin1') as f:
            content = f.read()

        # Parse email content
        msg = email.message_from_string(content)

        # Get subject and body
        subject = msg.get('Subject', '')

        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                ctype = part.get_content_type()
                cdispo = str(part.get('Content-Disposition'))

                # Skip attachments
                if ctype == 'text/plain' and 'attachment' not in cdispo:
                    part_body = part.get_payload(decode=True)
                    if part_body:
                        body = part_body.decode('latin1', errors='ignore')
                    break
        else:
            part_body = msg.get_payload(decode=True)
            if part_body:
                body = part_body.decode('latin1', errors='ignore')

        # Clean text - remove excessive whitespace
        body = re.sub(r'\s+', ' ', body).strip() if body else ""
        subject = re.sub(r'\s+', ' ', subject).strip()

        return subject, body
    except Exception as e:
        return "", ""

# Process Enron emails
print("Processing Enron emails...")
MAX_LEN = 128
#enron_dir = '/content/drive/MyDrive/email_bert/datasets/enron_mail_20150507/maildir/'
email_pairs = []

# Limit to a sample to avoid processing too many emails
users = os.listdir(enron_dir)[:30]  # Take first 30 users

for user in users:
    user_dir = os.path.join(enron_dir, user)
    if os.path.isdir(user_dir):
        for folder in os.listdir(user_dir)[:5]:  # Limit folders per user
            folder_path = os.path.join(user_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path)[:50]:  # Limit files per folder
                    file_path = os.path.join(folder_path, file)
                    if os.path.isfile(file_path):
                        subject, body = extract_email_content(file_path)

                        # Skip empty emails
                        if not body:
                            continue

                        # Create sentence pairs for NSP
                        # 1. Split email body into paragraphs
                        paragraphs = body.split('\n')
                        paragraphs = [p.strip() for p in paragraphs if p.strip()]

                        if len(paragraphs) < 2:
                            # If no clear paragraphs, split by sentences
                            sentences = re.split(r'[.!?]+', body)
                            sentences = [s.strip() for s in sentences if len(s.strip()) > 20]

                            if len(sentences) >= 2:
                                # Create pairs from consecutive sentences
                                for i in range(len(sentences) - 1):
                                    # Positive example: consecutive sentences
                                    first = sentences[i]
                                    second = sentences[i + 1]

                                    if len(first.split()) > 3 and len(second.split()) > 3:
                                        email_pairs.append([
                                            ' '.join(first.split()[:MAX_LEN]),
                                            ' '.join(second.split()[:MAX_LEN])
                                        ])
                        else:
                            # Create pairs from paragraphs
                            for i in range(len(paragraphs) - 1):
                                # Positive example: consecutive paragraphs
                                first = paragraphs[i]
                                second = paragraphs[i + 1]

                                if len(first.split()) > 3 and len(second.split()) > 3:
                                    email_pairs.append([
                                        ' '.join(first.split()[:MAX_LEN]),
                                        ' '.join(second.split()[:MAX_LEN])
                                    ])

print(f"Created {len(email_pairs)} email pairs")
# Sample one pair to verify
print("Sample pair:", email_pairs[20] if len(email_pairs) > 20 else email_pairs[0])

# Save pairs to Drive for reuse
import pickle
with open('/content/drive/MyDrive/email_bert/datasets/email_pairs.pkl', 'wb') as f:
    pickle.dump(email_pairs, f)

# Save data as txt file for tokenizer training
text_data = []
file_count = 0

for sample in tqdm.tqdm([x[0] for x in email_pairs] + [x[1] for x in email_pairs]):
    text_data.append(sample)

    # Save to file once we hit the 10K mark
    if len(text_data) == 10000:
        with open(f'/content/drive/MyDrive/email_bert/data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

# Save remaining data
if text_data:
    with open(f'/content/drive/MyDrive/email_bert/data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
        fp.write('\n'.join(text_data))

paths = [str(x) for x in Path('/content/drive/MyDrive/email_bert/data').glob('**/*.txt')]
print(f"Created {len(paths)} text files for tokenizer training")

# Train tokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=2,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
)

# Save tokenizer to Drive
# Save tokenizer
tokenizer.save_model('/content/drive/MyDrive/email_bert/tokenizer', 'bert-email')

# Load tokenizer (CORRECT way)
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast(
    vocab_file="/content/drive/MyDrive/email_bert/tokenizer/bert-email-vocab.txt",
    lowercase=True,
    strip_accents=False
)

# Test tokenizer
sample_text = "This is an email about the project deadline tomorrow."
token_ids = tokenizer(sample_text)['input_ids']
print("Sample tokenization:", tokenizer.convert_ids_to_tokens(token_ids))


import os
import math
import random
import itertools
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import tqdm
import wandb

# =====================================
# 1) BERTDataset (same interface as before)
# =====================================

class BERTDataset(Dataset):
    def __init__(self, data_pair, tokenizer, seq_len=256):
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.corpus_lines = len(data_pair)
        self.lines = data_pair

    def __len__(self):
        return self.corpus_lines

    def __getitem__(self, item):
        # Step 1: get email pair, either negative or positive (for NSP)
        t1, t2, is_next_label = self.get_sent(item)

        # Step 2: replace random words with mask / random words (for MLM)
        t1_random, t1_label = self.random_word(t1)
        t2_random, t2_label = self.random_word(t2)

        # Step 3: Add special tokens (CLS, SEP)
        cls_id = self.tokenizer.vocab['[CLS]']
        sep_id = self.tokenizer.vocab['[SEP]']
        pad_id = self.tokenizer.vocab['[PAD]']

        t1 = [cls_id] + t1_random + [sep_id]
        t2 = t2_random + [sep_id]

        t1_label = [pad_id] + t1_label + [pad_id]
        t2_label = t2_label + [pad_id]

        # Step 4: combine parts and add padding
        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        bert_input = (t1 + t2)[:self.seq_len]
        bert_label = (t1_label + t2_label)[:self.seq_len]

        padding_len = self.seq_len - len(bert_input)
        if padding_len > 0:
            padding = [pad_id] * padding_len
            bert_input.extend(padding)
            bert_label.extend(padding)
            segment_label.extend(padding)

        output = {
            "bert_input": bert_input,
            "bert_label": bert_label,
            "segment_label": segment_label,
            "is_next": is_next_label
        }

        return {key: torch.tensor(value) for key, value in output.items()}

    def random_word(self, sentence):
        """
        Very similar to your original logic:
        - 15% of tokens are selected for MLM
        - 80% -> [MASK]
        - 10% -> random token
        - 10% -> leave as is
        Labels:
        - token id where we predict
        - 0 where we don't predict (ignore_index=0 in MLM loss)
        """
        tokens = sentence.split()
        output_label = []
        output = []

        for token in tokens:
            prob = random.random()

            token_ids = self.tokenizer(token)['input_ids']
            # token_ids: [CLS, ..., SEP] or just [CLS, SEP] if weird
            if len(token_ids) <= 2:
                continue

            token_ids = token_ids[1:-1]  # remove special tokens

            if prob < 0.15:
                prob /= 0.15

                # 80%: replace with [MASK]
                if prob < 0.8:
                    for _ in range(len(token_ids)):
                        output.append(self.tokenizer.vocab['[MASK]'])
                # 10%: random token
                elif prob < 0.9:
                    for _ in range(len(token_ids)):
                        output.append(random.randrange(len(self.tokenizer.vocab)))
                # 10%: keep original
                else:
                    output.extend(token_ids)

                output_label.extend(token_ids)
            else:
                # no prediction for this token
                output.extend(token_ids)
                output_label.extend([0] * len(token_ids))

        # Already flat lists, but keep this in case
        output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
        output_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))

        return output, output_label

    def get_sent(self, index):
        """Return email segment pair - either positive or negative example for NSP."""
        t1, t2 = self.get_corpus_line(index)

        # 50% chance for positive / negative
        if random.random() > 0.5:
            return t1, t2, 1  # Positive example
        else:
            return t1, self.get_random_line(), 0  # Negative example

    def get_corpus_line(self, item):
        """Return segment pair from corpus."""
        return self.lines[item][0], self.lines[item][1]

    def get_random_line(self):
        """Return random segment."""
        i = random.randrange(len(self.lines))
        return self.lines[i][random.choice([0, 1])]


# ==========================
# 2) Embeddings & Encoder
# ==========================

class PositionalEmbedding(torch.nn.Module):
    def __init__(self, d_model, max_len=128):
        super().__init__()
        pe = torch.zeros(max_len, d_model).float()
        pe.requires_grad = False

        for pos in range(max_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                if i + 1 < d_model:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))

        # shape: (1, max_len, d_model)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: (batch_size, seq_len)
        seq_len = x.size(1)
        # Broadcast along batch dimension automatically
        return self.pe[:, :seq_len, :]


class BERTEmbedding(torch.nn.Module):
    """
    BERT Embedding:
      - TokenEmbedding
      - SegmentEmbedding
      - PositionalEmbedding
    """

    def __init__(self, vocab_size, embed_size, seq_len=256, dropout=0.1):
        super().__init__()
        self.embed_size = embed_size
        self.token = torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.segment = torch.nn.Embedding(3, embed_size, padding_idx=0)
        self.position = PositionalEmbedding(d_model=embed_size, max_len=seq_len)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, sequence, segment_label):
        # sequence, segment_label: (batch_size, seq_len)
        positions = self.position(sequence)         # (1, seq_len, d_model) -> broadcast
        x = self.token(sequence) + positions + self.segment(segment_label)
        return self.dropout(x)


class MultiHeadedAttention(torch.nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = torch.nn.Dropout(dropout)

        self.query = torch.nn.Linear(d_model, d_model)
        self.key = torch.nn.Linear(d_model, d_model)
        self.value = torch.nn.Linear(d_model, d_model)
        self.output_linear = torch.nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):
        # query/key/value: (batch_size, seq_len, d_model)
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)

        # (batch_size, seq_len, d_model) -> (batch_size, heads, seq_len, d_k)
        def split_heads(x):
            return x.view(x.size(0), -1, self.heads, self.d_k).permute(0, 2, 1, 3)

        query = split_heads(query)
        key = split_heads(key)
        value = split_heads(value)

        # scores: (batch_size, heads, seq_len, seq_len)
        scores = torch.matmul(query, key.permute(0, 1, 3, 2)) / math.sqrt(self.d_k)
        scores = scores.masked_fill(mask == 0, -1e9)

        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)

        # context: (batch_size, heads, seq_len, d_k)
        context = torch.matmul(weights, value)

        # -> (batch_size, seq_len, d_model)
        context = context.permute(0, 2, 1, 3).contiguous()
        context = context.view(context.size(0), -1, self.heads * self.d_k)

        return self.output_linear(context)


class FeedForward(torch.nn.Module):
    "Implements FFN: Linear -> GELU -> Dropout -> Linear"

    def __init__(self, d_model, middle_dim=2048, dropout=0.1):
        super().__init__()
        self.fc1 = torch.nn.Linear(d_model, middle_dim)
        self.fc2 = torch.nn.Linear(middle_dim, d_model)
        self.dropout = torch.nn.Dropout(dropout)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out


class EncoderLayer(torch.nn.Module):
    def __init__(
        self,
        d_model=768,
        heads=12,
        feed_forward_hidden=768 * 4,
        dropout=0.1
    ):
        super().__init__()
        self.layernorm1 = torch.nn.LayerNorm(d_model)
        self.layernorm2 = torch.nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadedAttention(heads, d_model, dropout=dropout)
        self.feed_forward = FeedForward(d_model, middle_dim=feed_forward_hidden, dropout=dropout)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, embeddings, mask):
        # embeddings: (batch_size, seq_len, d_model)
        # mask:       (batch_size, 1, 1, seq_len)

        attended = self.self_multihead(embeddings, embeddings, embeddings, mask)
        attended = self.dropout(attended)
        out1 = self.layernorm1(attended + embeddings)

        ff = self.feed_forward(out1)
        ff = self.dropout(ff)
        out2 = self.layernorm2(ff + out1)

        return out2


# ==========================
# 3) BERT backbone
# ==========================

class BERT(torch.nn.Module):
    """
    BERT model (encoder only).
    """

    def __init__(self, vocab_size, d_model=768, n_layers=10, heads=12, dropout=0.1, seq_len=64):
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # Slightly smaller FFN: 3 * d_model (instead of 4 * d_model)
        self.feed_forward_hidden = d_model * 3

        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=d_model, seq_len=seq_len, dropout=dropout)

        self.encoder_blocks = torch.nn.ModuleList(
            [
                EncoderLayer(
                    d_model=d_model,
                    heads=heads,
                    feed_forward_hidden=self.feed_forward_hidden,
                    dropout=dropout
                )
                for _ in range(n_layers)
            ]
        )

    def forward(self, x, segment_info):
        # x: (batch_size, seq_len)
        # segment_info: (batch_size, seq_len)

        # mask: (batch_size, 1, 1, seq_len)
        mask = (x > 0).unsqueeze(1).unsqueeze(2)

        # embedding: (batch_size, seq_len, d_model)
        x = self.embedding(x, segment_info)

        for encoder in self.encoder_blocks:
            x = encoder(x, mask)

        return x


# ==========================
# 4) Heads: NSP + MLM + BERTLM
# ==========================

class NextSentencePrediction(torch.nn.Module):
    def __init__(self, hidden):
        super().__init__()
        self.linear = torch.nn.Linear(hidden, 2)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # use [CLS] token embedding
        return self.softmax(self.linear(x[:, 0]))


class MaskedLanguageModel(torch.nn.Module):
    def __init__(self, hidden, vocab_size):
        super().__init__()
        self.linear = torch.nn.Linear(hidden, vocab_size)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # x: (batch_size, seq_len, hidden)
        return self.softmax(self.linear(x))


class BERTLM(torch.nn.Module):
    """
    Joint model: BERT backbone + NSP head + MLM head
    """

    def __init__(self, bert: BERT, vocab_size):
        super().__init__()
        self.bert = bert
        self.next_sentence = NextSentencePrediction(self.bert.d_model)
        self.mask_lm = MaskedLanguageModel(self.bert.d_model, vocab_size)

    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        return self.next_sentence(x), self.mask_lm(x)


# ==========================
# 5) Optimizer + Trainer
# ==========================

class ScheduledOptim:
    """
    Simple wrapper for learning rate scheduling (Noam-style).
    """

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps
        ])

    def _update_learning_rate(self):
        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr


class BERTTrainer:
    def __init__(
        self,
        model,
        train_dataloader,
        test_dataloader=None,
        lr=1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=800,          # changed from 10000 to 800
        log_freq=10,
        device='cuda'
    ):
        self.device = device
        self.model = model.to(device)
        self.train_data = train_dataloader
        self.test_data = test_dataloader

        # Optimizer + schedule
        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, self.model.bert.d_model, n_warmup_steps=warmup_steps
        )

        # SEPARATE LOSSES:
        # - NSP: no ignore_index
        # - MLM: ignore_index=0 (PAD / non-predicted tokens)
        self.nsp_criterion = torch.nn.NLLLoss()
        self.mlm_criterion = torch.nn.NLLLoss(ignore_index=0)

        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))

    def train(self, epoch):
        self.iteration(epoch, self.train_data, train=True)

    def test(self, epoch):
        if self.test_data is not None:
            self.iteration(epoch, self.test_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        avg_loss = 0.0
        avg_nsp_loss = 0.0
        avg_mlm_loss = 0.0
        total_correct = 0
        total_element = 0

        mode = "train" if train else "test"

        data_iter = tqdm.tqdm(
            enumerate(data_loader),
            desc=f"EP_{mode}:{epoch}",
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, batch in data_iter:
            # Move batch to device
            batch = {key: value.to(self.device) for key, value in batch.items()}

            # Forward
            next_sent_output, mask_lm_output = self.model(
                batch["bert_input"], batch["segment_label"]
            )

            # Losses
            nsp_loss = self.nsp_criterion(next_sent_output, batch["is_next"])
            mlm_loss = self.mlm_criterion(
                mask_lm_output.transpose(1, 2), batch["bert_label"]
            )
            loss = nsp_loss + mlm_loss

            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()

            # NSP accuracy
            correct = next_sent_output.argmax(dim=-1).eq(batch["is_next"]).sum().item()
            total_correct += correct
            total_element += batch["is_next"].nelement()

            avg_loss += loss.item()
            avg_nsp_loss += nsp_loss.item()
            avg_mlm_loss += mlm_loss.item()

            if i % self.log_freq == 0:
                post_fix = {
                    "epoch": epoch,
                    "iter": i,
                    "mode": mode,
                    "avg_loss": avg_loss / (i + 1),
                    "avg_nsp_loss": avg_nsp_loss / (i + 1),
                    "avg_mlm_loss": avg_mlm_loss / (i + 1),
                    "avg_acc": total_correct / total_element * 100,
                    "loss": loss.item()
                }
                data_iter.write(str(post_fix))

            # W&B logging occasionally
            if i % 300 == 0 and train:
                metrics = {
                    f"{mode}/avg_loss": avg_loss / (i + 1),
                    f"{mode}/avg_nsp_loss": avg_nsp_loss / (i + 1),
                    f"{mode}/avg_mlm_loss": avg_mlm_loss / (i + 1),
                    f"{mode}/avg_acc": total_correct / total_element * 100,
                }
                wandb.log(metrics)

        print(
            f"EP{epoch}, {mode}: "
            f"avg_loss={avg_loss / len(data_iter):.4f}, "
            f"nsp_loss={avg_nsp_loss / len(data_iter):.4f}, "
            f"mlm_loss={avg_mlm_loss / len(data_iter):.4f}, "
            f"total_acc={total_correct * 100.0 / total_element:.2f}"
        )


# ==========================
# 6) Save / load helpers
# ==========================

def save_model(model, optimizer, metrics, epoch, path):
    torch.save(
        {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'metric': metrics,
            'epoch': epoch
        },
        path
    )

def load_model(model, optimizer=None, path='./checkpoint.pth'):
    checkpoint = torch.load(path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    metrics = checkpoint['metric']
    return model, optimizer, epoch, metrics


# ==========================
# 7) Build dataset & dataloader (using your email_pairs + tokenizer)
# ==========================

MAX_LEN = 128  # same as before

train_data = BERTDataset(email_pairs, tokenizer, seq_len=MAX_LEN)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, pin_memory=True)

sample_batch = next(iter(train_loader))
print("Batch shapes:", {k: v.shape for k, v in sample_batch.items()})


# ==========================
# 8) Init W&B, Model, Trainer, Training Loop
# ==========================

# Login once at the start of notebook (you might have already done this)
# wandb.login()  # comment out if you don't want W&B tracking

run = wandb.init(
    name="bert-email-pretraining-small",
    project="bert-email-project",
)

print("Initializing smaller BERT model (~100M params)...")
vocab_size = len(tokenizer.vocab)

bert_model = BERT(
    vocab_size=vocab_size,
    d_model=768,
    n_layers=10,   # smaller than 12
    heads=12,
    dropout=0.1,
    seq_len=MAX_LEN
)

bert_lm = BERTLM(bert_model, vocab_size=vocab_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_trainer = BERTTrainer(
    bert_lm,
    train_loader,
    device=device,
    warmup_steps=800
)

# ==========================
# 9) Train
# ==========================

epochs = 5
ckpt_dir = '/content/drive/MyDrive/email_bert/checkpoints'
os.makedirs(ckpt_dir, exist_ok=True)

for epoch in range(epochs):
    bert_trainer.train(epoch)

    epoch_path = os.path.join(ckpt_dir, f'bert_email_epoch_{epoch}.pth')
    save_model(bert_lm, bert_trainer.optim, {"epoch": epoch}, epoch, epoch_path)
    print(f"Saved model after epoch {epoch} to {epoch_path}")

# Final model
final_path = '/content/drive/MyDrive/email_bert/bert_email_final_small.pth'
save_model(bert_lm, bert_trainer.optim, {"final_epoch": epochs}, epochs, final_path)
print(f"Pre-training complete. Final model saved to {final_path}")

wandb.finish()

print("\nModel and data files are saved to Google Drive in the following locations:")
print("- Dataset: /content/drive/MyDrive/email_bert/datasets/")
print("- Tokenizer: /content/drive/MyDrive/email_bert/tokenizer/")
print("- Checkpoints: /content/drive/MyDrive/email_bert/checkpoints/")
print("- Final model: /content/drive/MyDrive/email_bert/bert_email_final_small.pth")

Processing Enron emails...
Created 47680 email pairs
Sample pair: ['If you need anything just let me know', 'Thanks, Crystal AGENT CS/CS BOOKING REF Y4LBHX SCHWIEGER/JAMES ENRON 1400 SMITH HOUSTON TX 77002 DATE: APR 09 2001 SERVICE DATE FROM TO DEPART ARRIVE AIR FRANCE 08AUG PARIS COPENHAGEN 1250P 245P AF 2350 W WED CHARLES DE GAU COPENHAGEN APT EQP: BOEING 737-300 RESERVATION CONFIRMED AIR FRANCE 19AUG COPENHAGEN PARIS 105P 300P AF 2051 W SUN COPENHAGEN APT CHARLES DE GAU EQP: BOEING 737-500 RESERVATION CONFIRMED ===== Vitol Travel Services 1100 Louisiana Suite 3230 Houston, Texas 77002 Phone - 713-759-1444 Fax - 713-759-9006 __________________________________________________ Do You Yahoo']


100%|██████████| 95360/95360 [00:00<00:00, 727034.96it/s]


Created 16 text files for tokenizer training
Sample tokenization: ['[CLS]', 'this', 'is', 'an', 'email', 'about', 'the', 'project', 'deadline', 'tomorrow', '[UNK]', '[SEP]']
Batch shapes: {'bert_input': torch.Size([32, 128]), 'bert_label': torch.Size([32, 128]), 'segment_label': torch.Size([32, 128]), 'is_next': torch.Size([32])}


Initializing smaller BERT model (~100M params)...
Total Parameters: 105188402


EP_train:0:   0%|| 1/1490 [00:04<1:42:03,  4.11s/it]

...

{'epoch': 2, 'iter': 860, 'mode': 'train', 'avg_loss': 7.602005403631655, 'avg_nsp_loss': 0.6944120628770082, 'avg_mlm_loss': 6.907593341654602, 'avg_acc': 50.38472706155633, 'loss': 8.041398048400879}


EP_train:2:  58%|| 871/1490 [57:23<37:23,  3.62s/it]

{'epoch': 2, 'iter': 870, 'mode': 'train', 'avg_loss': 7.6018869770106985, 'avg_nsp_loss': 0.6944290998335136, 'avg_mlm_loss': 6.907457878887995, 'avg_acc': 50.35519517795637, 'loss': 7.749552249908447}


EP_train:2:  59%|| 879/1490 [57:55<40:16,  3.95s/it]


KeyboardInterrupt: 

In [ ]:
!ls /content
enron_dir = '/content/maildir/'

import os, re, email, tqdm

MAX_LEN = 128
email_pairs = []

def fast_body_extract(path):
    try:
        with open(path, "r", errors="ignore") as f:
            msg = email.message_from_string(f.read())
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    try: return part.get_payload(decode=True).decode("latin1")
                    except: return ""
        else:
            try: return msg.get_payload(decode=True).decode("latin1")
            except: return ""
    except:
        return ""

print("Processing emails...")

users = os.listdir(enron_dir)[:30]
for u in users:
    udir = os.path.join(enron_dir, u)
    if not os.path.isdir(udir): continue
    for folder in os.listdir(udir)[:5]:
        fdir = os.path.join(udir, folder)
        if not os.path.isdir(fdir): continue
        for file in os.listdir(fdir)[:50]:
            fpath = os.path.join(fdir, file)
            body = fast_body_extract(fpath)
            if not body: continue

            sents = re.split(r'[.!?]+', body)
            sents = [s.strip() for s in sents if len(s.strip()) > 10]

            for i in range(len(sents)-1):
                email_pairs.append([
                    " ".join(sents[i].split()[:MAX_LEN]),
                    " ".join(sents[i+1].split()[:MAX_LEN])
                ])

print("Pairs:", len(email_pairs))
from pathlib import Path

path = "/content/drive/MyDrive/email_bert/data/"
os.makedirs(path, exist_ok=True)

chunks = []
chunk_id = 0

for t1, t2 in tqdm.tqdm(email_pairs):
    chunks.append(t1)
    chunks.append(t2)
    if len(chunks) >= 10000:
        with open(f"{path}/text_{chunk_id}.txt", "w") as f:
            f.write("\n".join(chunks))
        chunks, chunk_id = [], chunk_id+1

if chunks:
    with open(f"{path}/text_{chunk_id}.txt", "w") as f:
        f.write("\n".join(chunks))

paths = [str(x) for x in Path(path).glob("*.txt")]
print("Text files:", len(paths))
from tokenizers import BertWordPieceTokenizer

tok = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tok.train(
    files=paths,
    vocab_size=30000,
    min_frequency=2,
    limit_alphabet=1000,
    special_tokens=["[PAD]","[CLS]","[SEP]","[MASK]","[UNK]"]
)

tok.save_model("/content/drive/MyDrive/email_bert/tokenizer","bert-email")

from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast(
    vocab_file="/content/drive/MyDrive/email_bert/tokenizer/bert-email-vocab.txt",
    lowercase=True,
    strip_accents=False
)

print("Tokenizer size:", len(tokenizer))
import torch
from torch.utils.data import Dataset

class EmailBERTDataset(Dataset):
    def __init__(self, pairs, tokenizer, seq_len=256):
        self.pairs = pairs
        self.tok = tokenizer
        self.seq_len = seq_len
        self.vocab_size = len(tokenizer)

    def __len__(self): return len(self.pairs)

    def __getitem__(self, i):
        t1, t2 = self.pairs[i]

        # 50% random NSP replacement
        if torch.rand(1).item() < 0.5:
            is_next = 1
        else:
            t2 = self.pairs[ torch.randint(len(self.pairs),(1,)).item() ][1]
            is_next = 0

        out = self.tok(
            t1, t2,
            max_length=self.seq_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        ids = out["input_ids"].squeeze()
        seg = out["token_type_ids"].squeeze()

        # Create MLM mask
        mask_prob = 0.15
        rand = torch.rand(ids.shape)

        mask = (rand < mask_prob) & (ids != self.tok.pad_token_id) & (ids != self.tok.cls_token_id) & (ids != self.tok.sep_token_id)
        labels = ids.clone()
        labels[~mask] = -100

        # Replace masked tokens
        ids_masked = ids.clone()
        ids_masked[mask] = self.tok.mask_token_id

        return {
            "input_ids": ids_masked,
            "token_type_ids": seg,
            "labels": labels,
            "is_next": torch.tensor(is_next)
        }
from torch.utils.data import DataLoader

train_dataset = EmailBERTDataset(email_pairs, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

batch = next(iter(train_loader))
print("Batch OK:", batch["input_ids"].shape)
import torch.nn as nn
import torch
from torch.cuda.amp import autocast, GradScaler

class BERTSmall(nn.Module):
    def __init__(self, vocab, d_model=512, layers=8, heads=8, seq_len=256):
        super().__init__()
        self.emb = nn.Embedding(vocab, d_model)
        self.pos = nn.Embedding(seq_len, d_model)
        self.seg = nn.Embedding(2, d_model)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=heads,
            dim_feedforward=d_model*3,
            batch_first=True,
            activation="gelu"
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=layers)

        self.mlm = nn.Linear(d_model, vocab)
        self.nsp = nn.Linear(d_model, 2)

    def forward(self, ids, seg):
        b, L = ids.shape
        pos = torch.arange(L, device=ids.device).unsqueeze(0)

        x = self.emb(ids) + self.pos(pos) + self.seg(seg)
        x = self.encoder(x)

        mlm_logits = self.mlm(x)
        nsp_logits = self.nsp(x[:,0])
        return mlm_logits, nsp_logits
class Trainer:
    def __init__(self, model, loader, device="cuda"):
        self.model = model.to(device)
        self.loader = loader
        self.device = device
        self.opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
        self.scaler = GradScaler()
        self.mlm_loss = nn.CrossEntropyLoss(ignore_index=-100)
        self.nsp_loss = nn.CrossEntropyLoss()

    import os

# -------------------------
# Count model parameters
# -------------------------
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# ==========================================================
#  UPDATED TRAINING FUNCTION WITH METRICS & CHECKPOINTS
# ==========================================================
def train_with_checkpoints(trainer, epochs=100, ckpt_dir="/content/drive/MyDrive/email_bert/checkpoints_small/"):
    os.makedirs(ckpt_dir, exist_ok=True)

    print("\n==============================")
    print("🚀 Training Started")
    print("Total Trainable Parameters:", count_parameters(trainer.model))
    print("==============================\n")

    for ep in range(epochs):
        total = 0
        correct = 0
        mlm_loss_sum = 0
        nsp_loss_sum = 0
        total_loss_sum = 0
        steps = 0

        for batch in tqdm.tqdm(trainer.loader, desc=f"Epoch {ep}"):
            ids = batch["input_ids"].to(trainer.device)
            seg = batch["token_type_ids"].to(trainer.device)
            mlm_labels = batch["labels"].to(trainer.device)
            nsp_labels = batch["is_next"].to(trainer.device)

            with autocast():
                mlm_logits, nsp_logits = trainer.model(ids, seg)
                loss_mlm = trainer.mlm_loss(
                    mlm_logits.view(-1, mlm_logits.size(-1)),
                    mlm_labels.view(-1)
                )
                loss_nsp = trainer.nsp_loss(nsp_logits, nsp_labels)
                loss = loss_mlm + loss_nsp

            trainer.opt.zero_grad()
            trainer.scaler.scale(loss).backward()
            trainer.scaler.step(trainer.opt)
            trainer.scaler.update()

            # Metrics
            correct += (nsp_logits.argmax(-1) == nsp_labels).sum().item()
            total += ids.size(0)

            mlm_loss_sum += loss_mlm.item()
            nsp_loss_sum += loss_nsp.item()
            total_loss_sum += loss.item()
            steps += 1

        # ---- END OF EPOCH METRICS ----
        avg_mlm = mlm_loss_sum / steps
        avg_nsp = nsp_loss_sum / steps
        avg_total = total_loss_sum / steps
        acc = correct / total * 100

        print(f"\n📊 Epoch {ep} Summary")
        print(f"  MLM Loss: {avg_mlm:.4f}")
        print(f"  NSP Loss: {avg_nsp:.4f}")
        print(f"  Total Loss: {avg_total:.4f}")
        print(f"  NSP Accuracy: {acc:.2f}%")
        print("------------------------------")

        # Save every 10 epochs
        if (ep + 1) % 10 == 0:
            save_path = os.path.join(ckpt_dir, f"bert_email_epoch_{ep}.pth")
            torch.save(trainer.model.state_dict(), save_path)
            print(f"💾 Saved checkpoint: {save_path}")

    # Save final model
    final_path = "/content/drive/MyDrive/email_bert/bert_email_83M_final.pth"
    torch.save(trainer.model.state_dict(), final_path)
    print(f"\n🎉 Final model saved at: {final_path}")



In [ ]:
train_with_checkpoints(trainer, epochs=100)


In [ ]:
!ls -R /content/drive
!ls -R /content/gdrive


In [ ]:
from google.colab import files

files.download('/content/drive/MyDrive/email_bert/checkpoints_small/bert_email_epoch_9.pth')
# or
#files.download('/content/drive/MyDrive/email_bert/bert_email_83M.pth')


In [ ]:
!ls -R /content/drive
!ls -R /content/gdrive


In [ ]:
!ls -lh /content/drive/MyDrive/email_bert/datasets


In [ ]:
!ls -l /content/drive/MyDrive/email_bert/tokenizer


In [ ]:
!ls /content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls -la /content/drive/MyDrive/email_bert/checkpoints/